In [40]:
import matplotlib.pyplot as plt
import numpy as np
import mxnet as mx
from mxnet import gluon, nd, image
from mxnet.gluon.data.vision import transforms
from gluoncv.data.transforms import video
from gluoncv import utils
from gluoncv.model_zoo import get_model

In [41]:
# from src.video import Video
# from gluoncv.utils.filesystem import try_import_decord

# decord = try_import_decord()
# v = Video('../data/abseiling_k400.mp4', decord)



In [42]:
# v.preview()

In [43]:
from pathlib import Path

TEST_VIDEO_URL = Path('volleyball/1567___555133a4b7a84ed7bd77bd424d225b23.mp4')

import cv2


cap = cv2.VideoCapture(TEST_VIDEO_URL.as_posix())

frames = []
while(cap.isOpened()):
    ret, frame = cap.read()
    frames.append(frame)
    if ret:
        cv2.imshow("frame", frame)
        cv2.waitKey(1)
    else:
        break

cap.release()
cv2.destroyAllWindows()

# cap.release()
# cv2.destroyAllWindows()
#
# frame_id_list = range(0, 64, 2)
# video_data = vr.get_batch(frame_id_list).asnumpy()
# clip_input = [frames[vid, :, :, :] for vid, _ in enumerate(frame_id_list)]
clip_input = frames[:64]

In [44]:
import numpy as np
from time import sleep
import cv2 as cv

for frame in clip_input:
    cv.imshow('frame',frame)
    if cv.waitKey(1) == ord('q'):
        break
    sleep(0.03)
# cap.release()
cv.destroyAllWindows()

In [45]:
transform_fn = video.VideoGroupValTransform(size=224, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
clip_input = transform_fn(clip_input)
clip_input = np.stack(clip_input, axis=0)
clip_input = clip_input.reshape((-1,) + (32, 3, 224, 224))
clip_input = np.transpose(clip_input, (0, 2, 1, 3, 4))
print('Video data is downloaded and preprocessed.')

Video data is downloaded and preprocessed.


In [46]:
model_name = 'i3d_inceptionv1_kinetics400'
net = get_model(model_name, nclass=400, pretrained=True)
print('%s model is successfully loaded.' % model_name)

i3d_inceptionv1_kinetics400 model is successfully loaded.


In [47]:
pred = net(nd.array(clip_input))

classes = net.classes
topK = 50
ind = nd.topk(pred, k=topK)[0].astype('int')
print('The input video clip is classified to be')
for i in range(topK):
    print('\t[%s], with probability %.3f.'%
          (classes[ind[i].asscalar()], nd.softmax(pred)[0][ind[i]].asscalar()))

The input video clip is classified to be
	[feeding_fish], with probability 0.448.
	[swimming_breast_stroke], with probability 0.267.
	[catching_fish], with probability 0.092.
	[swimming_butterfly_stroke], with probability 0.079.
	[swimming_backstroke], with probability 0.048.
	[snorkeling], with probability 0.024.
	[canoeing_or_kayaking], with probability 0.016.
	[cleaning_pool], with probability 0.004.
	[kitesurfing], with probability 0.003.
	[surfing_water], with probability 0.003.
	[jumping_into_pool], with probability 0.002.
	[crossing_river], with probability 0.002.
	[scuba_diving], with probability 0.002.
	[jetskiing], with probability 0.002.
	[water_sliding], with probability 0.001.
	[windsurfing], with probability 0.001.
	[petting_animal_-not_cat-], with probability 0.001.
	[water_skiing], with probability 0.001.
	[diving_cliff], with probability 0.001.
	[sailing], with probability 0.000.
	[somersaulting], with probability 0.000.
	[parasailing], with probability 0.000.
	[spring